# Example 3: Propagation of H2O vibration under polynomial PES

| run type      | wavefunction | backend | Basis  | steps |
| ---           | ---          | ---     | ---    | ---   |
| propagation | MPS-SM | Numpy   | HO-DVR | 5000    |

## 1. Import modules

- Required in **any** calculations

In [1]:
import platform
import sys

from pytdscf import BasInfo, Model, Simulator, __version__

print(
    f"pytdscf version: {__version__}, Python version: {sys.version}, platform: {platform.platform()}"
)

17:04:14 | INFO | Param defined: keys = {'enable_summed_op'}
17:04:14 | INFO | Param defined: verbose = 4
17:04:14 | INFO | Param defined: mass = 1.0
17:04:14 | INFO | Param defined: epsrho = 1e-08
17:04:14 | INFO | Param defined: tol_CMF = 1e-14
17:04:14 | INFO | Param defined: max_stepsize = 0.41341373335170156
17:04:14 | INFO | Param defined: tol_RK45 = 1e-08
17:04:14 | INFO | Param defined: load_balance_interval = 100
17:04:14 | INFO | Param defined: pytest_enabled = False
17:04:14 | INFO | Param defined: mpi_rank = 0
17:04:14 | INFO | Param defined: mpi_size = 1
17:04:14 | INFO | Param defined: mpi_comm = <mpi4py.MPI.Intracomm object at 0x7f219aa126d0>


pytdscf version: 1.0.5, Python version: 3.12.2 (main, Feb 25 2024, 04:38:01) [Clang 17.0.6 ], platform: Linux-5.15.167.4-microsoft-standard-WSL2-x86_64-with-glibc2.39


## 2. Set DVR primitive basis

In [2]:
from math import sqrt

import numpy as np
from discvar import HarmonicOscillator as HO

from pytdscf.dvr_operator_cls import TensorOperator
from pytdscf.hamiltonian_cls import TensorHamiltonian
from pytdscf.potentials.h2o_potential import k_orig

backend = "numpy"
ndim = 3
freqs = [sqrt(k_orig[(k, k)]) for k in range(1, ndim + 1)]  # a.u.

nprims = [9] * ndim  # Number of primitive basis

basis = [
    HO(nprim, omega, units="a.u.")
    for nprim, omega in zip(nprims, freqs, strict=True)
]
basinfo = BasInfo([basis])  # Set basis information object

**MPS-MCTDH wavefunction**
$$
|\Psi_{\rm{MPS-MCTDH}}\rangle = \sum_{\mathbf \{j\}}\sum_{\mathbf \{\tau\}}
a\substack{j_1 \\ 1\tau_1}a\substack{j_2 \\ \tau_1\tau_2} \cdots a\substack{j_f \\ \tau_{f-1}1}
|\varphi_{j_1}^{(1)}(q_1)\rangle|\varphi_{j_2}^{(2)}(q_2)\rangle
\cdots|\varphi_{j_f}^{(f)}(q_f)\rangle
$$
where SPF is 
$$
\varphi_{j_p}^{(p)}(q_p) = \sum_{i_p=1}^{n_p} c_{i_p}^{j_p}\chi_{i_p}^{(p)}(q_p) \; (j_p = 1,2,\ldots, N_p)
$$

Here, select $\{\chi_{i_p}^{(p)}(q_p)\}$ as Harmonic Oscillator eigenfunction.
See detail in [documenation](https://qclovers.github.io/PyTDSCF/pytdscf.html#pytdscf.primints_cls.poly_HO_FBR).
Here one define $n_p$ = 9, $N_p$ = 9. (Standard Method)

**NOTE**

- In MPS,  $n = N$ (SM) is usually better than $n < M$ (MCTDH).  Only when using a laptop, MCTDH may be better. (RAM required in MCTDH is smaller than SM.)

## 3. Set Hamiltonian (Taylor expansion polynomial PES in MPO format)

We have already constructed MPO in the previous relaxation step.

In [3]:
data = np.load("h2o_pot_mpo.npz")
pot_mpo = [data[f"W{i}"] for i in range(len(data.files))]
data = np.load("h2o_kin_mpo.npz")
kin_mpo = [data[f"W{i}"] for i in range(len(data.files))]

### Setup Hamiltonian

In [4]:
potential = [
    [
        {
            (tuple((i,) for i in range(0, ndim))): TensorOperator(mpo=pot_mpo),
            (tuple((i, i) for i in range(0, ndim))): TensorOperator(
                mpo=kin_mpo
            ),
        }
    ]
]

H = TensorHamiltonian(
    ndof=len(basis), potential=potential, kinetic=None, backend=backend
)

operators = {"hamiltonian": H}

17:04:14 | INFO | Start tensor decomposition: type = QRD


## 4. Set wavefunction (MPS) and All Model

- `m_aux_max` is MPS bond dimension (maximum of auxiliary index $\tau_p$)


In [5]:
model = Model(basinfo, operators)
model.m_aux_max = 9

## 5. Execute Calculation

- time step width is defined by `stepsize`=0.2 fs

In this calculation, one runs

- Real-time propagation

- Restart from $\hat{\mu}|\Psi_{\rm GS}\rangle$ wavefunction. (restart file suffix is `_dipole`)

F.Y.I., See also [documentation](https://qclovers.github.io/PyTDSCF/pytdscf.html#pytdscf.const_cls.Const.set_runtype)


**NOTE**

- Runtype cannnot rebind. If you change runtype, you should restart the kernel.

- JAX is better when simulating more large systems. (f>6, m>10)

- If `AVG Krylov iteration` in the log file is much larger than 5, you should set smaller timestep.

In [6]:
jobname = "h2o_polynomial"
simulator = Simulator(jobname, model, backend="numpy")
simulator.propagate(
    maxstep=1000,  # 200 fs
    stepsize=0.2,
    restart=True,
    savefile_ext="_prop",
    loadfile_ext="_dipole",
)  # i.e., 500 fs

17:04:14 | INFO | Param defined: jobname = h2o_polynomial_prop
17:04:14 | INFO | Log file is ./h2o_polynomial_prop/main.log
17:04:14 | INFO | Param defined: verbose = 2
17:04:14 | INFO | Param defined: doRestart = True
17:04:14 | INFO | Param defined: doRelax = False
17:04:14 | INFO | Param defined: doAppDipo = False
17:04:14 | INFO | Param defined: doDVR = True
17:04:14 | INFO | Param defined: savefile_ext = _prop
17:04:14 | INFO | Param defined: loadfile_ext = _dipole
17:04:14 | INFO | Param defined: time_fs_init = 0.0
17:04:14 | INFO | Param defined: maxstep = 1000
17:04:14 | INFO | Param defined: doOrtho = False
17:04:14 | INFO | Param defined: doTDHamil = False
17:04:14 | INFO | Param defined: oldcode = False
17:04:14 | INFO | Param defined: use_jax = False
17:04:14 | INFO | Param defined: thresh_exp = 1e-09
17:04:14 | INFO | Param defined: standard_method = True
17:04:14 | INFO | Param defined: use_mpo = True
17:04:14 | INFO | Param defined: adaptive = False
17:04:14 | INFO | Par

(0.020896110048546822, <pytdscf.wavefunction.WFunc at 0x7f21c041ca40>)

## 6. Check Log file
See `h2o_polynomial_prop/main.log`, which is defined as `jobname`.

In [7]:
!tail h2o_polynomial_prop/main.log

2025-06-11 17:04:40 | DEBUG | pytdscf.properties:_export_properties:383 - | autocorr: -0.9275+0.3656i| pop 1.0000 | ene[eV]:  0.5686121 | time[fs]:  198.800 | elapsed[sec]:    22.50 
2025-06-11 17:04:40 | DEBUG | pytdscf.properties:_export_properties:383 - | autocorr: -0.7493+0.6580i| pop 1.0000 | ene[eV]:  0.5686121 | time[fs]:  199.000 | elapsed[sec]:    22.52 
2025-06-11 17:04:40 | DEBUG | pytdscf.properties:_export_properties:383 - | autocorr: -0.4826+0.8729i| pop 1.0000 | ene[eV]:  0.5686121 | time[fs]:  199.200 | elapsed[sec]:    22.54 
2025-06-11 17:04:40 | DEBUG | pytdscf.properties:_export_properties:383 - | autocorr: -0.1589+0.9849i| pop 1.0000 | ene[eV]:  0.5686121 | time[fs]:  199.400 | elapsed[sec]:    22.56 
2025-06-11 17:04:40 | DEBUG | pytdscf.properties:_export_properties:383 - | autocorr:  0.1837+0.9808i| pop 1.0000 | ene[eV]:  0.5686121 | time[fs]:  199.600 | elapsed[sec]:    22.58 
2025-06-11 17:04:40 | INFO | pytdscf.simulator_cls:_execute:379 - Saved wavefunction 

/home/hinom/.local/share/uv/python/cpython-3.12.2-linux-x86_64-gnu/lib/python3.12/pty.py:95: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
